# NebulaGraph

>[NebulaGraph](https://www.nebula-graph.io/) 是一个开源的、分布式的、可扩展的、闪电般的图数据库，专为超大规模图构建，延迟毫秒级。它使用 `nGQL` 图查询语言。
>
>[nGQL](https://docs.nebula-graph.io/3.0.0/3.ngql-guide/1.nGQL-overview/1.overview/) 是 `NebulaGraph` 的声明式图查询语言。它支持富有表现力和高效的图模式。`nGQL` 面向开发者和运维专业人士设计。`nGQL` 是一种类似 SQL 的查询语言。

本笔记本展示了如何使用 LLM 为 `NebulaGraph` 数据库提供自然语言接口。

## 配置

您可以通过运行以下脚本将 `NebulaGraph` 集群作为 Docker 容器启动：

```bash
curl -fsSL nebula-up.siwei.io/install.sh | bash
```

其他选项包括：
- 作为 [Docker Desktop Extension](https://www.docker.com/blog/distributed-cloud-native-graph-database-nebulagraph-docker-extension/) 安装。请参阅此处的[文档](https://docs.nebula-graph.io/3.5.0/2.quick-start/1.quick-start-workflow/)
- NebulaGraph Cloud 服务。请参阅此处的[文档](https://www.nebula-graph.io/cloud)
- 通过软件包、源代码或 Kubernetes 进行部署。请参阅此处的[文档](https://docs.nebula-graph.io/)

集群运行后，我们就可以为数据库创建 `SPACE` 和 `SCHEMA` 了。

In [ ]:
%pip install --upgrade --quiet  ipython-ngql
%load_ext ngql

# connect ngql jupyter extension to nebulagraph
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula
# create a new space
%ngql CREATE SPACE IF NOT EXISTS langchain(partition_num=1, replica_factor=1, vid_type=fixed_string(128));

In [ ]:
# Wait for a few seconds for the space to be created.
%ngql USE langchain;

创建模式，有关完整数据集的参考请访问[此处](https://www.siwei.io/en/nebulagraph-etl-dbt/)。

In [ ]:
%%ngql
CREATE TAG IF NOT EXISTS movie(name string);
CREATE TAG IF NOT EXISTS person(name string, birthdate string);
CREATE EDGE IF NOT EXISTS acted_in();
CREATE TAG INDEX IF NOT EXISTS person_index ON person(name(128));
CREATE TAG INDEX IF NOT EXISTS movie_index ON movie(name(128));

等待 schema 创建完成，然后我们就可以插入一些数据。

In [ ]:
%%ngql
INSERT VERTEX person(name, birthdate) VALUES "Al Pacino":("Al Pacino", "1940-04-25");
INSERT VERTEX movie(name) VALUES "The Godfather II":("The Godfather II");
INSERT VERTEX movie(name) VALUES "The Godfather Coda: The Death of Michael Corleone":("The Godfather Coda: The Death of Michael Corleone");
INSERT EDGE acted_in() VALUES "Al Pacino"->"The Godfather II":();
INSERT EDGE acted_in() VALUES "Al Pacino"->"The Godfather Coda: The Death of Michael Corleone":();

In [1]:
from langchain.chains import NebulaGraphQAChain
from langchain_community.graphs import NebulaGraph
from langchain_openai import ChatOpenAI

In [2]:
graph = NebulaGraph(
    space="langchain",
    username="root",
    password="nebula",
    address="127.0.0.1",
    port=9669,
    session_pool_size=30,
)

## 刷新图谱 schema 信息

如果数据库的 schema 发生变化，您可以刷新生成 nGQL 语句所需的 schema 信息。

In [ ]:
# graph.refresh_schema()

In [3]:
print(graph.get_schema)

Node properties: [{'tag': 'movie', 'properties': [('name', 'string')]}, {'tag': 'person', 'properties': [('name', 'string'), ('birthdate', 'string')]}]
Edge properties: [{'edge': 'acted_in', 'properties': []}]
Relationships: ['(:person)-[:acted_in]->(:movie)']



## 查询图谱

我们现在可以使用图谱的 Cypher QA 链来提问了

In [5]:
chain = NebulaGraphQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [6]:
chain.run("Who played in The Godfather II?")



> Entering new NebulaGraphQAChain chain...
Generated nGQL:
MATCH (p:`person`)-[:acted_in]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather II'
RETURN p.`person`.`name`
Full Context:
{'p.person.name': ['Al Pacino']}

> Finished chain.


'Al Pacino played in The Godfather II.'